# Team win history GRU

Choose two teams, the history will first be displayed as (for example) 0,1,0,..,1,0 etc.
Then these are compiled and a GRU is made to predict the probability of future win of that team against a challenger...

In [1]:
# Python imports
import pandas as pd
import numpy as np

In [2]:
dat = pd.read_csv('matchdata.csv',encoding = "ISO-8859-1")


/home/max/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (82,83,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Need a function that pulls out only a certain teams data:

def team_data(team): # probably by index...
    df = dat.loc[(dat.ix[:,3] == team) |  (dat.ix[:,4] == team) ]
    return(df)


In [258]:
def win_series(team1,team2):
    wins = []
    losses = []
    
    # find only games where team 1 and team 2 are playing
    df = dat.loc[((dat.ix[:,3] == team1) |  (dat.ix[:,4] == team1) ) & ((dat.ix[:,3] == team2) |  (dat.ix[:,4] == team2) )]
    df = df.reset_index(drop = True) 
    # now for each row find if they won or lost: winning team == team1
    for i in range(len(df)):

        if (df.ix[i,5] == team1):
            # they won
            wins.append(1)
            losses.append(0)
        elif (df.ix[i,5] == team2):
            # they lost
            wins.append(0)
            losses.append(1)
        else:
            # can't find them
            wins.append('?')
    
    # now reverse the win vector to be oldest to newest
    wins = np.flipud(wins)
    ins= wins[:len(wins)-1]
    targs = wins[1:len(wins)] 
    lins = np.array(losses[:len(wins)-1]) 
    ltargs = np.array(losses[1:len(wins)])
    
    return ins, targs, lins,ltargs
    

In [247]:
''' sequence padding code '''
from __future__ import absolute_import
# -*- coding: utf-8 -*-
import random
from six.moves import range


def pad_sequences(sequences, maxlen=None, dtype='int32',
                  padding='pre', truncating='pre', value=0.):
    '''Pads each sequence to the same length:
    the length of the longest sequence.

    If maxlen is provided, any sequence longer
    than maxlen is truncated to maxlen.
    Truncation happens off either the beginning (default) or
    the end of the sequence.

    Supports post-padding and pre-padding (default).

    # Arguments
        sequences: list of lists where each element is a sequence
        maxlen: int, maximum length
        dtype: type to cast the resulting sequence.
        padding: 'pre' or 'post', pad either before or after each sequence.
        truncating: 'pre' or 'post', remove values from sequences larger than
            maxlen either in the beginning or in the end of the sequence
        value: float, value to pad the sequences to the desired value.

    # Returns
        x: numpy array with dimensions (number_of_sequences, maxlen)
    '''
    lengths = [len(s) for s in sequences]

    nb_samples = len(sequences)
    if maxlen is None:
        maxlen = np.max(lengths)

    # take the sample shape from the first non empty sequence
    # checking for consistency in the main loop below.
    sample_shape = tuple()
    for s in sequences:
        if len(s) > 0:
            sample_shape = np.asarray(s).shape[1:]
            break

    x = (np.ones((nb_samples, maxlen) + sample_shape) * value).astype(dtype)
    for idx, s in enumerate(sequences):
        if len(s) == 0:
            continue  # empty list was found
        if truncating == 'pre':
            trunc = s[-maxlen:]
        elif truncating == 'post':
            trunc = s[:maxlen]
        else:
            raise ValueError('Truncating type "%s" not understood' % truncating)

        # check `trunc` has expected shape
        trunc = np.asarray(trunc, dtype=dtype)
        if trunc.shape[1:] != sample_shape:
            raise ValueError('Shape of sample %s of sequence at position %s is different from expected shape %s' %
                             (trunc.shape[1:], idx, sample_shape))

        if padding == 'post':
            x[idx, :len(trunc)] = trunc
        elif padding == 'pre':
            x[idx, -len(trunc):] = trunc
        else:
            raise ValueError('Padding type "%s" not understood' % padding)
    return x


In [259]:
X, T, XL , TL = win_series('EnVyUs','fnatic')
X1, T1, XL1 , TL1 = win_series('EnVyUs','Natus Vincere')
X2, T2, XL2 , TL2 = win_series('EnVyUs','Luminosity')
X3, T3, XL3 , TL3 = win_series('EnVyUs','Virtus.pro')
X4, T4, XL4 , TL4 = win_series('EnVyUs','TSM')
X5, T5, XL5 , TL5 = win_series('EnVyUs','Astralis')
X6, T6, XL6 , TL6 = win_series('EnVyUs','NiP')


# now zero pad all to be the same shape:



# combine into an array:

ins =  pad_sequences(np.array([X.tolist(),X1.tolist(),X2.tolist(),X3.tolist(),X4.tolist(),X5.tolist(),X6.tolist()]) )
targs = pad_sequences(np.array([T.tolist(),T1.tolist(),T2.tolist(),T3.tolist(),T4.tolist(),T5.tolist(),T6.tolist()]) )

lins =  pad_sequences(np.array([XL.tolist(),XL1.tolist(),XL2.tolist(),XL3.tolist(),XL4.tolist(),XL5.tolist(),XL6.tolist()]) )

ltargs = pad_sequences(np.array([TL.tolist(),TL1.tolist(),TL2.tolist(),TL3.tolist(),TL4.tolist(),TL5.tolist(),TL6.tolist()]) )



In [249]:
''' split into testing and training sets '''
len(ins)
split = 0.7
Ntrain = int(round(len(ins)*split,0))

train_ins = ins[:Ntrain]
test_ins = ins[Ntrain:]
train_T = targs[:Ntrain]
test_T = targs[Ntrain:]

In [239]:
# Run the model of GRU
# Keras imports 
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM,GRU, Embedding

In [244]:
in_neurons = 1
hidden_neurons = 1
 
model = Sequential()  
model.add(GRU(output_dim=hidden_neurons, input_dim= in_neurons, return_sequences=True, consume_less='gpu')) 
model.compile(loss="binary_crossentropy", optimizer= 'rmsprop' ) 

In [245]:
''' train this fella '''
model.fit(np.reshape(train_ins,(len(train_ins),np.shape(train_ins)[1],1) ), np.reshape(train_T,(len(train_ins),np.shape(train_ins)[1],1) ), batch_size=1, nb_epoch = 100, validation_split=0.1)  


Train on 4 samples, validate on 1 samples
Epoch 1/100
4/4 [==============================] - 0s - loss: 16.5830 - val_loss: 10.8487
Epoch 2/100
4/4 [==============================] - 0s - loss: 16.5830 - val_loss: 10.8487
Epoch 3/100
4/4 [==============================] - 0s - loss: 16.5830 - val_loss: 10.8487
Epoch 4/100
4/4 [==============================] - 0s - loss: 16.5830 - val_loss: 10.8487
Epoch 5/100
4/4 [==============================] - 0s - loss: 16.5830 - val_loss: 10.8487
Epoch 6/100
4/4 [==============================] - 0s - loss: 16.5830 - val_loss: 10.8487
Epoch 7/100
4/4 [==============================] - 0s - loss: 16.5830 - val_loss: 10.8487
Epoch 8/100
4/4 [==============================] - 0s - loss: 16.5830 - val_loss: 10.8487
Epoch 9/100
4/4 [==============================] - 0s - loss: 16.5830 - val_loss: 10.8487
Epoch 10/100
4/4 [==============================] - 0s - loss: 16.5830 - val_loss: 10.8487
Epoch 11/100
4/4 [==============================] - 0s - 

In [209]:
Tshape = np.shape(train_ins)[1]
predicted = model.predict(np.reshape(test_ins,(np.shape(test_T)[0],Tshape,1)))

preds = pd.DataFrame()
preds['Real'] = np.reshape(test_T[1],(Tshape) )
preds['Predicted'] = np.reshape(predicted[1],(Tshape) )
print(preds)

    Real  Predicted
0      0   0.007264
1      0   0.013209
2      0   0.018086
3      0   0.022092
4      0   0.025389
5      0   0.028105
6      0   0.030345
7      0   0.032193
8      0   0.033720
9      0   0.034981
10     0   0.036024
11     0   0.036886
12     0   0.543951
13     0   0.497786
14     1   0.452059
15     0   0.546196
16     0   0.500032
17     1   0.454264
18     1   0.546123
19     1   0.542265
20     1   0.542456
21     0   0.542447
22     1   0.496282
23     0   0.544554
24     0   0.498389
25     0   0.452651
26     1   0.408179
27     1   0.547465
28     1   0.542207
29     1   0.542459
30     0   0.542447
31     1   0.496282
32     1   0.544554
33     1   0.542342
34     0   0.542453
35     0   0.496288
36     1   0.450590
37     1   0.546245
38     1   0.542259
39     0   0.542457
40     0   0.496292
41     0   0.450594
42     1   0.406198
43     0   0.547514
44     0   0.501351
45     0   0.455560
46     0   0.410982
47     0   0.368370
48     0   0.328332


In [227]:
INP = win_series('EnVyUs','G2')
predicted = model.predict(np.reshape(np.array(INP) , (2,len(INP[0]),1)) )[0]


In [228]:
predicted

array([[ 0.00726403],
       [ 0.01320932],
       [ 0.54285729],
       [ 0.54242665],
       [ 0.54244822],
       [ 0.54244715],
       [ 0.49628255],
       [ 0.45058441],
       [ 0.40618879],
       [ 0.36383224],
       [ 0.32410753],
       [ 0.5488987 ],
       [ 0.54221153],
       [ 0.54245901],
       [ 0.54244661],
       [ 0.49628198]], dtype=float32)

In [234]:
INP = win_series('EnVyUs','HellRaisers')

predicted = model.predict(np.reshape(np.array(INP) , (2,len(INP[0]),1)) )[0]
TTshape = len(predicted)
preds = pd.DataFrame()
preds['Real'] = np.reshape(INP[0],(TTshape) )
preds['Predicted'] = np.reshape(predicted,(TTshape) )
print(preds)

    Real  Predicted
0      1   0.542201
1      1   0.542459
2      1   0.542447
3      1   0.542447
4      1   0.542447
5      0   0.496283
6      0   0.450584
7      1   0.546245
8      1   0.542259
9      0   0.496094
10     1   0.544562
11     1   0.542341
12     1   0.542453
13     1   0.542447
14     1   0.542447
